In [1]:
import json
import openai
from pprint import pprint
from dotenv import load_dotenv
import os
import pandas as pd
import random
import requests

TRAIN_DATA_FILE = "train.json"
PROMPT_FILE = "prompt.txt"

/Users/williamfang/cs41-env/lib/python3.11/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
load_dotenv()
API_KEY = os.getenv('API_KEY')

In [3]:
def get_user_prompt(data_point): 
    pass

def get_system_prompt(data_point):
    pass

In [ ]:
def call_openai_api(data_point):
    system_prompt = get_system_prompt(data_point)
    user_prompt = get_user_prompt(data_point)
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json",
    }
    data = {
        "model": "gpt-4-turbo",
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}]
    }
    response = requests.post(url, headers=headers, json=data)
    return response.json()